In [1]:
import os, sys, glob

import argparse
parser = argparse.ArgumentParser()
# Dataset
parser.add_argument('--set', type=str, default='valid')
# Model/Config
parser.add_argument('--step', type=str, default='050000')
parser.add_argument('--ckpt_selector', type=str, default='ema')
parser.add_argument('--cfg_name', type=str, default=None)
parser.add_argument('--log_dir', type=str, default=None)
# Interpolation
parser.add_argument('--interpolate', nargs='+', default=None)
parser.add_argument('--interpolate_step', type=int, default=15)
parser.add_argument('--interpolate_noise', action='store_true', default=False)
parser.add_argument('--lerp', action='store_true', default=False)
parser.add_argument('--slerp', action='store_true', default=False)
parser.add_argument('--uncond_sampling', action='store_true', default=False)
parser.add_argument('--uncond_sampling_iters', type=int, default=1)
parser.add_argument('--reverse_sampling', action='store_true', default=False)
parser.add_argument('--separate_reverse_sampling', action='store_true', default=False)
# Samples selection
parser.add_argument('--n_subject', type=int, default=-1)
parser.add_argument('--sample_pair_json', type=str, default=None)
parser.add_argument('--sample_pair_mode', type=str, default=None)
parser.add_argument('--src_dst', nargs='+', default=[])
# Pertubation the image condition
parser.add_argument('--perturb_img_cond', action='store_true', default=False)
parser.add_argument('--perturb_mode', type=str, default='zero')
parser.add_argument('--perturb_where', nargs='+', default=[])

# Rendering
parser.add_argument('--render_mode', type=str, default="shape")
parser.add_argument('--rotate_normals', action='store_true', default=False)
# Diffusion
parser.add_argument('--diffusion_steps', type=int, default=1000)
parser.add_argument('--denoised_clamp', type=float, default=None)
# Misc.
parser.add_argument('--seed', type=int, default=23)
parser.add_argument('--gpu_id', type=str, default="0")
parser.add_argument('--save_intermediate', action='store_true', default=False)
parser.add_argument('--postfix', type=str, default='')
parser.add_argument('--ovr_img', type=str, default=None)
parser.add_argument('--ovr_mod', action='store_true', default=False)
parser.add_argument('--norm_img', action='store_true', default=False)
parser.add_argument('--use_global_norm', action='store_true', default=False)
parser.add_argument('--norm_space', type=str, default='rgb')

args = parser.parse_args(args=[])

import os, sys, glob
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch as th
import PIL, cv2
import json
import copy
import time
import torchvision
import pytorch_lightning as pl
import scipy
sys.path.insert(0, '../../../')
from guided_diffusion.script_util import (
    seed_all,
)

from guided_diffusion.tensor_util import (
    make_deepcopyable,
    dict_slice,
)

import torch
import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms.functional as F

plt.rcParams["savefig.bbox"] = 'tight'
def show(imgs, figsize=(8, 6)):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(ncols=len(imgs), squeeze=False, figsize=figsize)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

from guided_diffusion.dataloader.img_deca_datasets import load_data_img_deca

# Sample utils
sys.path.insert(0, '../../')
from sample_utils import (
    ckpt_utils, 
    params_utils, 
    vis_utils, 
    file_utils, 
    inference_utils, 
    mani_utils,
)
device = 'cuda' if th.cuda.is_available() and th._C._cuda_getDeviceCount() > 0 else 'cpu'

In [2]:
seed_all(47)

################ SETTINGS ################
args.cfg_name = "Masked_Face_woclip+UNet_BgNoHead_share_dpm_noise_masking+shadow.yaml"
args.log_dir = "Masked_Face_woclip+UNet_BgNoHead_share_dpm_noise_masking+shadow"
# args.cfg_name = "Masked_Face_woclip+UNet_Bg_share_dpm_noise_masking+shadow.yaml"
# args.log_dir = "Masked_Face_woclip+UNet_Bg_share_dpm_noise_masking+shadow"
# args.cfg_name = "Masked_Face_woclip+UNet_Bg_share_dpm_noise_masking.yaml"
# args.log_dir = "Masked_Face_woclip+UNet_Bg_share_dpm_noise_masking"
# args.cfg_name = "Face+UNet_Bg_share_dpm_noise_masking+shadow.yaml"
# args.log_dir = "Face+UNet_Bg_share_dpm_noise_masking+shadow"

# args.cfg_name = "Masked_Face_woclip+UNet_Bg_share_dpm_noise_masking_shadow_256.yaml"
# args.log_dir = "Masked_Face_woclip+UNet_Bg_share_dpm_noise_masking_shadow_256_cont"

args.step = '050000'
args.ckpt_selector = 'ema'
args.set = 'valid'
# args.sample_pair_json = './sample_json/itw_samples.json'
# args.sample_pair_json = './sample_json/ipynb_samples.json'
args.sample_pair_json = './sample_json/debug_multipie-Moke.json'
args.sample_pair_mode = 'pair'
dataset = 'mp'
# dataset = 'itw'

# Load Ckpt
if args.cfg_name is None:
    args.cfg_name = args.log_dir + '.yaml'
ckpt_loader = ckpt_utils.CkptLoader(log_dir=args.log_dir, cfg_name=args.cfg_name)
cfg = ckpt_loader.cfg

print(f"[#] Sampling with diffusion_steps = {args.diffusion_steps}")
cfg.diffusion.diffusion_steps = args.diffusion_steps
model_dict, diffusion = ckpt_loader.load_model(ckpt_selector=args.ckpt_selector, step=args.step)
model_dict = inference_utils.eval_mode(model_dict)

# Load dataset

if dataset == 'itw':
    img_dataset_path = f"/data/mint/DPM_Dataset/ITW/itw_images_aligned/"
    deca_dataset_path = f"/data/mint/DPM_Dataset/ITW/params/"
    img_ext = '.png'
    cfg.dataset.training_data = 'ITW'
    cfg.dataset.data_dir = f'{cfg.dataset.root_path}/{cfg.dataset.training_data}/itw_images_aligned/'
elif dataset == 'ffhq':
    img_dataset_path = f"/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/"
    deca_dataset_path = f"/data/mint/DPM_Dataset/ffhq_256_with_anno/params/"
    img_ext = '.jpg'
    cfg.dataset.training_data = 'ffhq_256_with_anno'
    cfg.dataset.data_dir = f'{cfg.dataset.root_path}/{cfg.dataset.training_data}/ffhq_256/'
elif dataset == 'mp':
    img_dataset_path = f"/data/mint/DPM_Dataset/MultiPIE/mp_aligned/"
    deca_dataset_path = f"/data/mint/DPM_Dataset/MultiPIE/params/"
    img_ext = '.png'
    cfg.dataset.training_data = 'MultiPIE'
    cfg.dataset.data_dir = f'{cfg.dataset.root_path}/{cfg.dataset.training_data}/mp_aligned/'
else: raise NotImplementedError

cfg.dataset.deca_dir = f'{cfg.dataset.root_path}/{cfg.dataset.training_data}/params/'
cfg.dataset.face_segment_dir = f"{cfg.dataset.root_path}/{cfg.dataset.training_data}/face_segment/"
cfg.dataset.deca_rendered_dir = f"{cfg.dataset.root_path}/{cfg.dataset.training_data}/rendered_images/"
cfg.dataset.laplacian_mask_dir = f"{cfg.dataset.root_path}/{cfg.dataset.training_data}/eyes_segment/"
cfg.dataset.laplacian_dir = f"{cfg.dataset.root_path}/{cfg.dataset.training_data}/laplacian/"

loader, dataset, avg_dict = load_data_img_deca(
    data_dir=img_dataset_path,
    deca_dir=deca_dataset_path,
    batch_size=int(1e7),
    image_size=cfg.img_model.image_size,
    deterministic=cfg.train.deterministic,
    augment_mode=cfg.img_model.augment_mode,
    resize_mode=cfg.img_model.resize_mode,
    in_image_UNet=cfg.img_model.in_image,
    params_selector=cfg.param_model.params_selector,
    rmv_params=cfg.param_model.rmv_params,
    set_=args.set,
    cfg=cfg,
    mode='sampling',
    img_ext=img_ext,
)

# DECA Rendering
if np.any(['deca_masked' in n for n in list(filter(None, dataset.condition_image))]):
    mask = params_utils.load_flame_mask()
else: 
    mask=None
deca_obj = params_utils.init_deca(mask=mask)

[#] Model Path : 
#0 : /data/mint/model_logs_mount/v7/Masked_Face_woclip+UNet_BgNoHead_share_dpm_noise_masking+shadow/
[#] Config Path :  ['/home/mint/guided-diffusion/config/CVPRs/Final_Candidates/Masked_Face_woclip+UNet_BgNoHead_share_dpm_noise_masking+shadow.yaml']
Merging with :  Namespace(cfg='/home/mint/guided-diffusion/config/CVPRs/Final_Candidates/Masked_Face_woclip+UNet_BgNoHead_share_dpm_noise_masking+shadow.yaml')

[#] Sampling with diffusion_steps = 1000
[#] Available ckpt :  ['_000000.pt', '_000000.pt', '_010000.pt', '_010000.pt', '_020000.pt', '_020000.pt', '_030000.pt', '_030000.pt', '_040000.pt', '_040000.pt', '_050000.pt', '_050000.pt', '_060000.pt', '_060000.pt', '_070000.pt', '_070000.pt', '_080000.pt', '_080000.pt', '_090000.pt', '_090000.pt', '_100000.pt', '_100000.pt', '_110000.pt', '_110000.pt', '_120000.pt', '_120000.pt', '_130000.pt', '_130000.pt', '_140000.pt', '_140000.pt', '_150000.pt', '_150000.pt', '_160000.pt', '_160000.pt', '_170000.pt', '_170000.pt', '_

Loading deca params...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.88it/s]


[#] Bounding the input of UNet to +-1.0
[#] Parameters Conditioning
Params keys order :  ['shape', 'pose', 'exp', 'cam', 'faceemb', 'shadow']
Remove keys :  []
Input Image :  ['raw']
Image condition :  ['deca_masked_face_images_woclip']
DPM Image condition :  ['faceseg_nohead']
creating the FLAME Decoder
[#] DECA : SRenderY applied mask


In [3]:
data_size = dataset.__len__()
img_path = file_utils._list_image_files_recursively(f"{img_dataset_path}/{args.set}")

denoised_fn = None
pl_sampling = inference_utils.PLSampling(model_dict=model_dict, 
                                            diffusion=diffusion, 
                                            reverse_fn=diffusion.ddim_reverse_sample_loop, 
                                            forward_fn=diffusion.ddim_sample_loop,
                                            denoised_fn=denoised_fn,
                                            cfg=cfg,
                                            args=args)

def rgb_to_gray(img):
    out = (img[:, 0:1] * 0.2989) + (img[:, 1:2] * 0.5870) + (img[:, 2:3] * 0.1140)
    return out

In [4]:
def inversion(dat, norm_img, cond, norm_space='gray', sdedit=None):
    '''
    :param dat: images in B x C x H x W
    '''
    cond['use_cond_xt_fn'] = False
    cond['dpm_cond_img'] = None 
    
    if norm_img:
        print(f"[#] Normalize image in {norm_space}...")
       
        gray = rgb_to_gray(dat)
        sd, mu = th.std_mean(gray, dim=(1, 2, 3), keepdims=True)
        # print(sd, mu, gray.shape, dat.shape)
        # print(sd.shape, mu.shape, gray.shape, dat.shape)
        mu_d, sd_d = 114.49997340551313, 58.050383371049826
        mu_d = (mu_d / 127.5) - 1
        sd_d = (sd_d / 127.5)
        
        img = (dat - mu) / sd
        img = (img * sd_d) + mu_d
        
        #img = dat + 0.2
        img = th.clip(img, -1, 1)
        #show([(img[0] + 1) * 0.5, (dat[0] + 1) * 0.5])
        #return None
    else:
        img = dat

    # Reverse   
    reverse_ddim_sample = pl_sampling.reverse_proc(x=img, model_kwargs=cond, store_intermediate=False)
    noise_map = reverse_ddim_sample['final_output']['sample']
    # Forward   
    sample_ddim = pl_sampling.forward_proc(noise=noise_map, model_kwargs=cond, store_intermediate=False, sdedit=sdedit)
    
    out = sample_ddim['final_output']['sample']
    if norm_img:
        print("[#] DeNormalize image...")

        out = (out - mu_d) / sd_d
        out = (out * sd) + mu
        
    sample_frames = (out + 1) * 0.5
    
    return sample_frames   

def reverse(dat, norm_img, cond, norm_space='gray'):
    '''
    :param dat: images in 1 x C x H x W
    '''
    
    assert dat.shape[0] == 1
    if norm_img:
        print(f"[#] Normalize image in {norm_space}...")
       
        gray = rgb_to_gray(dat)
        sd, mu = th.std_mean(gray)
        mu_d, sd_d = 114.49997340551313, 58.050383371049826
        mu_d = (mu_d / 127.5) - 1
        sd_d = (sd_d / 127.5)
        
        img = (dat - mu) / sd
        img = (img * sd_d) + mu_d
        
        #img = dat + 0.2
        img = th.clip(img, -1, 1)
        #show([(img[0] + 1) * 0.5, (dat[0] + 1) * 0.5])
        #return None
        nf = {'mu':mu, 'sd':sd, 'mu_d':mu_d, 'sd_d':sd_d}
    else:
        img = dat
        nf = {}

    # Reverse   
    reverse_ddim_sample = pl_sampling.reverse_proc(x=img, model_kwargs=cond, store_intermediate=False)
    noise_map = reverse_ddim_sample['final_output']['sample']
        
    return noise_map, nf 
    
def forward(noise_map, cond, norm_img, nf):
    
    # Forward   
    sample_ddim = pl_sampling.forward_proc(noise=noise_map, model_kwargs=cond, store_intermediate=False)
    
    out = sample_ddim['final_output']['sample']
    if norm_img:
        print("[#] DeNormalize image...")
        out = (out - nf['mu_d']) / nf['sd_d']
        out = (out * nf['sd']) + nf['mu']
        
    sample_frames = (out + 1) * 0.5
    
    return sample_frames   

# Manipulate the condition

In [5]:
def make_condition(cond, src_idx, dst_idx, n_step=2, itp_func=None):
    condition_img = list(filter(None, dataset.condition_image))
    misc = {'condition_img':condition_img,
            'src_idx':src_idx,
            'dst_idx':dst_idx,
            'n_step':n_step,
            'avg_dict':avg_dict,
            'dataset':dataset,
            'args':args,
            'itp_func':itp_func,
            'img_size':cfg.img_model.image_size,
            'deca_obj':deca_obj,
            'cfg':cfg
            }  
    
    if itp_func is not None:
        cond['use_render_itp'] = False 
    else:
        cond['use_render_itp'] = True
        
    # This is for the noise_dpm_cond_img
    if cfg.img_model.apply_dpm_cond_img:
        cond['image'] = th.stack([cond['image'][src_idx]] * n_step, dim=0)
        for k in cfg.img_model.dpm_cond_img:
            cond[f'{k}_mask'] = th.stack([cond[f'{k}_mask'][src_idx]] * n_step, dim=0)
        
    cond, clip_ren = inference_utils.build_condition_image(cond=cond, misc=misc)
    cond = inference_utils.prepare_cond_sampling(cond=cond, cfg=cfg, use_render_itp=True)
    cond['cfg'] = cfg
    if (cfg.img_model.apply_dpm_cond_img) and (np.any(n is not None for n in cfg.img_model.noise_dpm_cond_img)):
        cond['use_cond_xt_fn'] = True
        for k, p in zip(cfg.img_model.dpm_cond_img, cfg.img_model.noise_dpm_cond_img):
            cond[f'{k}_img'] = cond[f'{k}_img'].to(device)
            if p is not None:
                if 'dpm_noise_masking' in p:
                    cond[f'{k}_mask'] = cond[f'{k}_mask'].to(device)
                    cond['image'] = cond['image'].to(device)
    

    if 'render_face' in args.interpolate:
        interp_set = args.interpolate.copy()
        interp_set.remove('render_face')
        
    # Interpolate non-spatial
    interp_cond = mani_utils.iter_interp_cond(cond, interp_set=interp_set, src_idx=src_idx, dst_idx=dst_idx, n_step=n_step, interp_fn=itp_func)
    cond.update(interp_cond)
        
    # Repeated non-spatial
    repeated_cond = mani_utils.repeat_cond_params(cond, base_idx=src_idx, n=n_step, key=mani_utils.without(cfg.param_model.params_selector, args.interpolate + ['light', 'img_latent']))
    cond.update(repeated_cond)

    # Finalize the cond_params
    cond = mani_utils.create_cond_params(cond=cond, key=mani_utils.without(cfg.param_model.params_selector, cfg.param_model.rmv_params))
    if cfg.img_cond_model.override_cond != '':
        to_tensor_key = ['cond_params'] + cfg.param_model.params_selector + [cfg.img_cond_model.override_cond]
    else:    
        to_tensor_key = ['cond_params'] + cfg.param_model.params_selector
    cond = inference_utils.to_tensor(cond, key=to_tensor_key, device=ckpt_loader.device)
    
    return cond
    

# Background

In [6]:
def face_part(mask, part='faceseg_head'):
    mask = np.array(mask)
    bg = (mask == 0)
    skin = (mask == 1)
    l_brow = (mask == 2)
    r_brow = (mask == 3)
    l_eye = (mask == 4)
    r_eye = (mask == 5)
    eye_g = (mask == 6)
    l_ear = (mask == 7)
    r_ear = (mask == 8)
    ear_r = (mask == 9)
    nose = (mask == 10)
    mouth = (mask == 11)
    u_lip = (mask == 12)
    l_lip = (mask == 13)
    neck = (mask == 14)
    neck_l = (mask == 15)
    cloth = (mask == 16)
    hair = (mask == 17)
    hat = (mask == 18)
    face = np.logical_or.reduce((skin, l_brow, r_brow, l_eye, r_eye, eye_g, l_ear, r_ear, ear_r, nose, mouth, u_lip, l_lip))
    if part == 'faceseg_head':
        seg_m = (face | neck | hair)
    return th.tensor(seg_m)

def replace_bg(image_name, dilate=None, rpl=None, vis=False, imgsize=128):
    mask = PIL.Image.open(f"/data/mint/DPM_Dataset/ffhq_256_with_anno/face_segment/valid/anno/anno_{image_name}.png")
    img = PIL.Image.open(f"/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/valid/{image_name}.jpg")

    # Load to [0, 1] image
    mask = face_part(mask)
    org_mask = mask.clone()
    if dilate is not None:
        from scipy import ndimage
        print("Apply dilation = ", dilate)
        mask = ndimage.binary_dilation(mask.cpu().numpy(), iterations=dilate)
        mask = th.tensor(mask)
        
    img = torchvision.transforms.ToTensor()(img) * 2 - 1
    rpl_mode = list(rpl.keys())[0]
    rpl_val = list(rpl.values())[0]
    
    if rpl_mode == 'constant':
        print(f"[#] Replace bg with {rpl_val}")
        rpl_bg = (th.randn_like(img) * 0 + rpl_val['val'])
    elif rpl_mode == 'balance_rgb':
        m_f, m_bg = mask!=0, mask==0
        assert (m_f + m_bg) == 256**2
        n_f, n_bg, n_pixel = th.sum(m_f)*3, th.sum(m_bg)*3, (256**2)*3
        mu_f =  th.sum(m_f * img) / n_f
        val = ((0 * n_pixel) - (n_f * mu_f)) / (n_bg)
        val = th.clip(val, -1, 1)
        rpl_bg = (th.randn_like(img) * 0 + val)
        print(f"[#] Replace bg with {val}")
    elif rpl_mode == 'mu_sd':
        rpl_bg = (th.randn_like(img) * rpl_val['sd'] + rpl_val['mu'])
    elif rpl_mode == 'image':
        bg_img = PIL.Image.open(f"/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/valid/{rpl_val}")
        bg_img = torchvision.transforms.ToTensor()(bg_img) * 2 - 1
        rpl_bg = bg_img
    elif rpl_mode == 'balance_Ndist':
        mu_d = 114.49997340551313 / 127.5 - 1
        sd = rpl_val['sd']
        m_f, m_bg = mask!=0, mask==0
        n_f, n_bg, n_pixel = th.sum(m_f)*3, th.sum(m_bg)*3, (256**2)*3
        mu_f =  th.sum(m_f * img) / n_f
        val = ((mu_d * n_pixel) - (n_f * mu_f)) / (n_bg)
        print(f"[#] Replace bg with Ndist({val}, {rpl_val['sd']}), clip={rpl_val['clip']}")
        rpl_bg = (th.normal(mean=val, std=sd, size=img.shape))
        if rpl_val['clip']:
            rpl_bg = th.clip(rpl_bg, -1, 1)
    else: raise NotImplementedError
    
    mod_img = img * (mask != 0) + rpl_bg * (mask == 0)
    mod_img = torchvision.transforms.Resize((imgsize, imgsize))(mod_img)
    bg_img = img * (mask == 0) + rpl_bg * (mask != 0)
    bg_img = torchvision.transforms.Resize((imgsize, imgsize))(bg_img)
    mask = torchvision.transforms.Resize((imgsize, imgsize), interpolation=PIL.Image.NEAREST)(mask[None])
    org_mask = torchvision.transforms.Resize((imgsize, imgsize), interpolation=PIL.Image.NEAREST)(org_mask[None])
    img = torchvision.transforms.Resize((imgsize, imgsize))(img)
    
    print('Mean : ', th.mean(mod_img))
    
    if vis:
        show(torchvision.utils.make_grid((mod_img + 1) * 0.5), figsize=(4, 8))
    return img, mod_img, bg_img, mask, org_mask 


# Relighting

In [7]:
def relight(dat, model_kwargs, norm_img, n_step=3, sidx=0, didx=1):
    show(torchvision.utils.make_grid((dat + 1) * 0.5), figsize=(4, 8))
    # Rendering
    args.interpolate = ['render_face']
    cond = copy.deepcopy(model_kwargs)
    cond = make_condition(cond=cond, 
                        src_idx=sidx, dst_idx=didx, 
                        n_step=n_step, itp_func=mani_utils.slerp)

    # Reverse 
    if cfg.img_cond_model.apply:
        cond_rev = copy.deepcopy(cond)
        cond_rev['cond_img'] = cond_rev['cond_img'][0:1, ...]
        cond_rev = pl_sampling.forward_cond_network(model_kwargs=cond_rev)
        cond_rev = dict_slice(in_d=cond_rev, keys=cond_rev.keys(), n=1)
        if cfg.img_model.conditioning:
            cond_rev['cond_params'] = cond_rev['cond_params'][0:1, ...]
        
    reverse_ddim_sample = pl_sampling.reverse_proc(x=dat[0:1, ...], model_kwargs=cond_rev, store_mean=True)
    noise_map = reverse_ddim_sample['final_output']['sample']
    rev_mean = reverse_ddim_sample['intermediate']
    
    #NOTE: rev_mean WILL BE MODIFIED; This is for computing the ratio of inversion (brightness correction).
    sample_ddim = pl_sampling.forward_proc(
        noise=noise_map,
        model_kwargs=cond_rev,
        store_intermediate=False,
        rev_mean=rev_mean)

    # Relight!
    cond['use_render_itp'] = True
    if cfg.img_cond_model.apply:
        cond_relight = pl_sampling.forward_cond_network(model_kwargs=cond)
        
    assert noise_map.shape[0] == 1
    rev_mean_first = [x[:1] for x in rev_mean]
    
    relight_out = pl_sampling.forward_proc(
        noise=th.repeat_interleave(noise_map, repeats=cond_relight["cond_img"].shape[0], dim=0),
        model_kwargs=cond_relight,
        store_intermediate=False,
        add_mean=rev_mean_first)
    return relight_out["final_output"]["sample"]

In [8]:
# assert False
all_img_idx, all_img_name, args.n_subject = mani_utils.get_samples_list(args.sample_pair_json, 
                                                                        args.sample_pair_mode, 
                                                                        args.src_dst, img_path, 
                                                                        args.n_subject)

# for idx in [1, 2, 3, 4]:
for idx in [149, 0]:
# for idx in range(len(all_img_idx)):
    img_idx = all_img_idx[idx]
    img_name = all_img_name[idx]
    n_step = 3

    dat = th.utils.data.Subset(dataset, indices=img_idx)
    subset_loader = th.utils.data.DataLoader(dat, batch_size=2,
                                        shuffle=False, num_workers=24)
                                
    # Load image & condition                        
    # for i in ['60182.jpg', '60008.jpg', '68782.jpg', -1.0, -0.5, 0.0, 0.5, 1.0]:
    dat, model_kwargs = list(iter(subset_loader))[0]
    d_tmp = []
    # rpl_bg = [None, {'image':model_kwargs['image_name'][0]}, {'balance_Ndist':{'sd':0.1, 'clip':True}}, {'balance_Ndist':{'sd':0.5, 'clip':True}}, {'balance_Ndist':{'sd':58.050383371049826/127.5, 'clip':True}}] 
    # rpl_bg = [None, {'balance_Ndist':{'sd':0.1, 'clip':True}} , {'balance_Ndist':{'sd':58.050383371049826/127.5, 'clip':True}}]
    # rpl_bg = [{'balance_Ndist':{'sd':0.1, 'clip':True}}]
    rpl_bg = [None]
    all_output = []
    for i in rpl_bg:
        print("Replacing : ", i)
        if i is not None:
            d, fg, bg, m, om = replace_bg(image_name=model_kwargs['image_name'][0].split('.')[0], dilate=10, rpl=i, imgsize=128)
            dat = fg[None][0:1, ...].cuda()
            model_kwargs['faceseg_bg_noface&nohair_mask'] = ~m[None]
            model_kwargs['original'] = d[None]
            model_kwargs['original_mask'] = om[None]
        else:
            dat = dat[0:1, ...].cuda()

        #TODO: Override the model_kwargs argument for 'mask'=m, 'image'=dat, 'bg'=bg
        # print(model_kwargs.keys())
        model_kwargs['faceseg_bg_noface&nohair_img'] = dat
        model_kwargs['image'] = dat
        
        # show(torchvision.utils.make_grid(th.clip((model_kwargs['faceseg_bg_noface&nohair_img']+1)*0.5, 0, 1)), figsize=(10, 15))
        # show(torchvision.utils.make_grid(model_kwargs['faceseg_bg_noface&nohair_mask']*1.0), figsize=(10, 15))
        # # ['faceseg_bg_noface&nohair_img'] = m[None]
        # print(model_kwargs['faceseg_bg_noface&nohair_img'].shape, model_kwargs['faceseg_bg_noface&nohair_mask'].shape)
        # print(bg.shape, fg.shape, m.shape)
        # assert False
        
        
        out = relight(dat=dat.clone(), model_kwargs=model_kwargs, n_step=3, norm_img=False)
        out = th.cat(((dat+1) * 0.5, (out+1) * 0.5), dim=0)
        all_output.append(torchvision.utils.make_grid(out))
    all_output = th.clip(th.cat((all_output), dim=1), 0, 1)
    show(all_output, figsize=(15, 15))
    

IndexError: list index out of range

# Laplacian Blending

In [ ]:
def create_gaussian_pyramid(img, level=5):
  pys = [img]
  for i in range(level-1):
    img = cv2.GaussianBlur(img, (5, 5), 0)
    img = cv2.resize(img, (int(img.shape[1] / 2), int(img.shape[0] / 2)), interpolation=cv2.INTER_LINEAR)
    pys.append(img)
  return pys


def create_laplacian_pyramid(img, level=5):
  pys = create_gaussian_pyramid(img, level)
  for i in range(level-1):
    pys[i] = pys[i] - cv2.resize(pys[i+1], (pys[i].shape[1], pys[i].shape[0]))
  return pys

def laplacian_blending(imgs, masks, level):
  summask = masks[0] + masks[1] + 1e-10
  img_lp = [None] * 2
  mask_lp = [None] * 2

  for i in range(2):
    img_lp[i] = create_laplacian_pyramid(imgs[i], level)
    mask_lp[i] = create_gaussian_pyramid(masks[i] / summask, level)

  output_lp = []
  for i in range(len(img_lp[0])):
    output_lp.append((img_lp[0][i] * mask_lp[0][i] + img_lp[1][i] * mask_lp[1][i]))

  output_lp = output_lp[::-1]
  prev_lvl = output_lp[0]
  for idx in range(len(output_lp)-1):
    prev_lvl = cv2.resize(prev_lvl, dsize=(output_lp[idx+1].shape[1], output_lp[idx+1].shape[0]))
    prev_lvl += output_lp[idx+1]

  return prev_lvl


show(torchvision.utils.make_grid(out), figsize=(10, 10))
show(torchvision.utils.make_grid(model_kwargs['faceseg_bg_noface&nohair_mask']*1.0), figsize=(3, 3))
show(torchvision.utils.make_grid(model_kwargs['original_mask']*1.0), figsize=(3, 3))
show(torchvision.utils.make_grid(out * ~model_kwargs['faceseg_bg_noface&nohair_mask'].cuda()), figsize=(10, 10))
show(torchvision.utils.make_grid(((model_kwargs['original']+1)*0.5) * model_kwargs['faceseg_bg_noface&nohair_mask']), figsize=(3, 3))
show(torchvision.utils.make_grid((model_kwargs['faceseg_bg_noface&nohair_img']+1)*0.5), figsize=(3, 3))
show(torchvision.utils.make_grid((model_kwargs['original']+1)*0.5), figsize=(3, 3))

out_blended = []
for i in range(out.shape[0]):
  img1 = out[i].cpu().numpy().transpose(1, 2, 0) * 255.0
  img1 = np.clip(img1, 0, 255)
  img2 = ((model_kwargs['original'][0] + 1)*127.5).cpu().numpy().transpose(1, 2, 0)
  img2 = np.clip(img2, 0, 255)
  #NOTE: Use Dilate-Erosion mask
  # mask = scipy.ndimage.binary_erosion(model_kwargs['faceseg_bg_noface&nohair_mask'][0].permute(1, 2, 0), iterations=5)
  # mask = scipy.ndimage.binary_dilation(model_kwargs['faceseg_bg_noface&nohair_mask'][0].permute(1, 2, 0).cpu().numpy(), iterations=4)
  # mask = th.tensor(mask).permute(2, 0, 1)[None]
  #NOTE: Use original mask
  mask = ~model_kwargs['original_mask']
  mask = th.repeat_interleave(mask, repeats=3, dim=1)[0].cpu().numpy().transpose(1, 2, 0)
  ovl_mask = th.logical_or(model_kwargs['original_mask'], model_kwargs['faceseg_bg_noface&nohair_mask'])

  plt.imshow(np.concatenate((img1, img2, mask*255, ((~mask*255)*0)+1), axis=1).astype(np.uint8))
  plt.show()
  # blended_out = laplacian_blending(imgs=[img1, img2], masks=[~mask, (mask*0)+1], level=2)
  blended_out = laplacian_blending(imgs=[img1, img2], masks=[~mask, mask], level=3)
  # plt.imshow(np.clip(blended_out,0, 255).astype(np.uint8))
  # plt.show()
  out_blended.append(th.tensor(blended_out).permute(2, 0, 1))
out_blended = (th.stack(out_blended, dim=0))

show(torchvision.utils.make_grid(out), figsize=(15, 15))
show(torchvision.utils.make_grid(th.clip(out_blended/255.0, 0, 1)), figsize=(15, 15))

# Alpha Bledning

In [ ]:
# show(torchvision.utils.make_grid(out), figsize=(10, 10))
# show(torchvision.utils.make_grid(model_kwargs['faceseg_bg_noface&nohair_mask']*1.0), figsize=(3, 3))
# show(torchvision.utils.make_grid(model_kwargs['original_mask']*1.0), figsize=(3, 3))
# show(torchvision.utils.make_grid(out * ~model_kwargs['faceseg_bg_noface&nohair_mask'].cuda()), figsize=(10, 10))

# show(torchvision.utils.make_grid(th.logical_or(model_kwargs['original_mask'], model_kwargs['faceseg_bg_noface&nohair_mask'])*1.0), figsize=(3, 3))
# show(torchvision.utils.make_grid(~th.logical_or(model_kwargs['original_mask'], model_kwargs['faceseg_bg_noface&nohair_mask'])*1.0), figsize=(3, 3))
# show(torchvision.utils.make_grid(((model_kwargs['original']+1)*0.5) * model_kwargs['faceseg_bg_noface&nohair_mask']), figsize=(3, 3))
# show(torchvision.utils.make_grid((model_kwargs['faceseg_bg_noface&nohair_img']+1)*0.5), figsize=(3, 3))
# show(torchvision.utils.make_grid((model_kwargs['original']+1)*0.5), figsize=(3, 3))

def alpha_blending(imgs, masks, ovl_mask, a=0.7):
    # Tensor
    # print(img1.shape, img2.shape, mask1.shape, mask2.shape, ovl_mask.shape)
    img1, img2 = imgs[0], imgs[1]
    mask1, mask2 = masks[0], masks[1]
    # border = ((a * img1) + ((1-a) * img2)) * ovl_mask
    border = ((ovl_mask * img1) + ((1-ovl_mask) * ~(ovl_mask==0) * img2))
    out = ((img1 * mask1) + (img2 * mask2)) + border
    # out = ((img1 * mask1))# + (img2 * mask2))
    return out
    

out_blended = []
for i in range(out.shape[0]):
  # print(th.max(out), th.min(out))
  # img1 = np.clip(img1, 0, 255)
  img1 = (out[i] / 0.5) - 1
  img2 = model_kwargs['original']
  # img2 = np.clip(img2, 0, 255)
  #NOTE: Use Dilate-Erosion mask
  # mask = scipy.ndimage.binary_erosion(model_kwargs['faceseg_bg_noface&nohair_mask'][0].permute(1, 2, 0), iterations=5)
  # mask = scipy.ndimage.binary_dilation(model_kwargs['faceseg_bg_noface&nohair_mask'][0].permute(1, 2, 0).cpu().numpy(), iterations=4)
  # mask = th.tensor(mask).permute(2, 0, 1)[None]
  #NOTE: Use original mask
  inner_m = model_kwargs['original_mask']
  inner_m = th.repeat_interleave(inner_m, repeats=3, dim=1)
  
  outer_m = model_kwargs['faceseg_bg_noface&nohair_mask']
  outer_m = th.repeat_interleave(outer_m, repeats=3, dim=1)
  
  ovl_mask = ~th.logical_or(model_kwargs['original_mask'], model_kwargs['faceseg_bg_noface&nohair_mask'])
  ovl_mask = th.repeat_interleave(ovl_mask, repeats=3, dim=1)
  
  
  ovl_mask = ovl_mask[0, 0:1].permute(1, 2, 0)
  ovl_mask = cv2.distanceTransform(src=ovl_mask[..., 0:1].cpu().numpy().astype(np.uint8), distanceType=cv2.DIST_L2, maskSize=3)[..., None]
  ovl_mask = cv2.normalize(ovl_mask, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)[..., None]
  ovl_mask = th.repeat_interleave(th.tensor(ovl_mask), repeats=3, dim=-1)
  ovl_mask = ovl_mask.permute(2, 0, 1)[None] / 255
  
  if i == 0:
    show(torchvision.utils.make_grid(inner_m*1.0), figsize=(5, 5))
    show(torchvision.utils.make_grid(outer_m*1.0), figsize=(5, 5))
    show(torchvision.utils.make_grid((outer_m+inner_m)*1.0), figsize=(5, 5))
    show(torchvision.utils.make_grid(ovl_mask*1.0), figsize=(5, 5))

  blended_out = alpha_blending(imgs=[img1.cuda(), img2.cuda()], masks=[inner_m.cuda(), outer_m.cuda()], ovl_mask=ovl_mask.cuda())
  out_blended.append(blended_out)
  
out_blended = (th.cat(out_blended, dim=0))
print(out_blended.shape)
print(th.max(out_blended), th.min(out_blended))
show(torchvision.utils.make_grid(th.clip((out_blended+1)*0.5, 0, 1)), figsize=(15, 15))

In [ ]:
show(torchvision.utils.make_grid(~ovl_mask*1.0), figsize=(5, 5))
x = (~ovl_mask[0].permute(1, 2, 0).cpu().numpy()*255).astype(np.uint8)
print(x.shape, x.dtype, np.max(x), np.min(x))
xd = cv2.distanceTransform(src=x[..., 0:1], distanceType=cv2.DIST_L2, maskSize=3)[..., None]
xd_norm = cv2.normalize(xd, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)[..., None]
print(np.max(xd_norm), np.min(xd_norm))

xd_norm = th.repeat_interleave(th.tensor(xd_norm), repeats=3, dim=-1)
show(torchvision.utils.make_grid(xd_norm.permute(2, 0, 1)[None]), figsize=(5, 5))
xd = th.repeat_interleave(th.tensor(xd), repeats=3, dim=-1)
show(torchvision.utils.make_grid(xd.permute(2, 0, 1)[None]), figsize=(5, 5))


gg = scipy.ndimage.distance_transform_edt(input=(~ovl_mask[0].permute(1, 2, 0).cpu().numpy()*255).astype(np.uint8))

show(torchvision.utils.make_grid(th.tensor(gg).permute(2, 0, 1)[None]), figsize=(5, 5))